# 常规赛：遥感影像地块分割baseline

### 比赛页面传送门： [常规赛：遥感影像地块分割](https://aistudio.baidu.com/aistudio/competition/detail/63)    欢迎一键三连 Fork~

### 赛题介绍
本赛题由 2020 CCF BDCI 遥感影像地块分割 初赛赛题改编而来。遥感影像地块分割, 旨在对遥感影像进行像素级内容解析，对遥感影像中感兴趣的类别进行提取和分类，在城乡规划、防汛救灾等领域具有很高的实用价值，在工业界也受到了广泛关注。现有的遥感影像地块分割数据处理方法局限于特定的场景和特定的数据来源，且精度无法满足需求。因此在实际应用中，仍然大量依赖于人工处理，需要消耗大量的人力、物力、财力。本赛题旨在衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果，利用人工智能技术，对多来源、多场景的异构遥感影像数据进行充分挖掘，打造高效、实用的算法，提高遥感影像的分析提取能力。
赛题任务
本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

### 数据说明
本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

### 训练数据集
样例图片及其标注如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/8087a965609d48a19a5e60f0330fa9054d04097644de48ffa3d557e7a8ad64ad)
![](https://ai-studio-static-online.cdn.bcebos.com/d18664ecf0514cb686c95958d30bbf8a2f5efb0691bc4d66a5f6317ab511d6d0)

![](https://ai-studio-static-online.cdn.bcebos.com/e42f2c222f204094ac3a0ea8582ca331b0452fb2b1704eabaae379d499906977)
![](https://ai-studio-static-online.cdn.bcebos.com/d5260bd5a820486a85aeb2105adfb6fa10284bd94453459f892755bc43e10b8a)


训练数据集文件名称：train_and_label.zip

包含2个子文件，分别为：训练数据集（原始图片）文件、训练数据集（标注图片）文件，详细介绍如下：

* **训练数据集**（原始图片）文件名称：img_train

	包含66,653张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，每张图片的名称形如T000123.jpg。
* **训练数据集**（标注图片）文件名称：lab_train

	包含66,653张分辨率为2m/pixel，尺寸为256 * 256的PNG图片，每张图片的名称形如T000123.png。
* **备注**： 全部PNG图片共包括4种分类，像素值分别为0、1、2、3。此外，像素值255为未标注区域，表示对应区域的所属类别并不确定，在评测中也不会考虑这部分区域。

### 测试数据集
测试数据集文件名称：img_test.zip，详细介绍如下：

包含4,609张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，文件名称形如123.jpg。、
### 数据增强工具
PaTTA：由第三方开发者组织AgentMaker维护的Test-Time Augmentation库，可在测试时通过数据增强方式产生额外的推理结果，在此基础上进行投票即可获得更稳定的成绩表现。 https://github.com/AgentMaker/PaTTA

RIFLE：由第三方开发者对ICML 2020中的《RIFLE: Backpropagation in Depth for Deep Transfer Learning through Re-Initializing the Fully-connected LayEr》论文所提供的封装版本，其通过对输出层多次重新初始化来使得深层backbone得到更充分的更新。 https://github.com/GT-ZhangAcer/RIFLE_Module

### 提交内容及格式
* 以zip压缩包形式提交结果文件，文件命名为 result.zip；
* zip压缩包中的图片格式必须为单通道PNG；
* PNG文件数需要与测试数据集中的文件数相同，且zip压缩包文件名需要与测试数据集中的文件名一一对应；
* 单通道PNG图片中的像素值必须介于0~3之间，像素值不能为255。如果存在未标注区域，评测系统会自动忽略对应区域的提交结果。
### 提交示例
提交文件命名为：result.zip，zip文件的组织方式如下所示：

```
主目录                                                                        
├── 1.png         #每个结果文件命名为：测试数据集图片名称+.png                      
├── 2.png                                                              
├── 3.png                                                    
├── ...     
```                                                
    
**备注**： 主目录中必须包含与测试数据集相同数目、名称相对应的单通道PNG图片，且每张单通道PNG图片中的像素值必须介于0~3之间，像素值不能为255。

### 环境安装

In [ ]:
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git

In [ ]:
# 安装所需依赖项
!pip install -r PaddleSeg/requirements.txt

### 解压数据集

In [ ]:
!unzip -q data/data80164/train_and_label.zip
!unzip -q data/data80164/img_test.zip

### 数据处理


In [ ]:
import os
import numpy as np

datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

total: 66652
img_train/T142629.jpg
lab_train/T142629.png
('img_train/T000289.jpg', 'lab_train/T000289.png')


In [ ]:
import numpy as np

# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地',  '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(5)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为训练集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))

train: 63320
valid: 3332


### 模型训练

### [PaddleSeg快速入门](https://github.com/PaddlePaddle/PaddleSeg/blob/release/v0.6.0/docs/usage.md)

> PaddleSeg/paddleseg/core/train.py

```
def train(model,
          train_dataset,
          val_dataset=None,
          optimizer=None,
          save_dir='output',
          iters=10000,
          batch_size=2,
          resume_model=None,
          save_interval=1000,
          log_iters=10,
          num_workers=0,
          use_vdl=False,
          losses=None,
          keep_checkpoint_max=5)
```

> PaddleSeg/configs/deeplabv3p/deeplabv3p_resnet50_os8_cityscapes_1024x512_80k.yml
```
_base_: '../_base_/cityscapes.yml'

batch_size: 8
iters: 20000

model:
  type: DeepLabV3P
  backbone:
    type: ResNet50_vd
    output_stride: 8
    multi_grid: [1, 2, 4]
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/resnet50_vd_ssld_v2.tar.gz
  num_classes: 4
  backbone_indices: [0, 3]
  aspp_ratios: [1, 12, 24, 36]
  aspp_out_channels: 256
  align_corners: False
  pretrained: null
```
> PaddleSeg/configs/_base_/cityscapes.yml
```
train_dataset:
  type: Dataset
  dataset_root: /home/aistudio
  train_path: /home/aistudio/train_list.txt
  num_classes: 4
  transforms:
    - type: ResizeStepScaling
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop
      crop_size: [1024, 512]
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/aistudio
  val_path: /home/aistudio/valid_list.txt
  num_classes: 4
  transforms:
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

learning_rate:
  value: 0.01
  decay:
    type: poly
    power: 0.9
    end_lr: 0.0

loss:
  types:
    - type: CrossEntropyLoss
  coef: [1]
```

In [ ]:
!python PaddleSeg/train.py \
        --config PaddleSeg/configs/deeplabv3p/deeplabv3p_resnet50_os8_cityscapes_1024x512_80k.yml \
        --use_vdl \
        --do_eval \
        --save_interval 10000 \
        --save_dir output

### 模型评估
> PaddleSeg/paddleseg/core/val.py
```
def evaluate(model,
             eval_dataset,
             aug_eval=False,
             scales=1.0,
             flip_horizontal=True,
             flip_vertical=False,
             is_slide=False,
             stride=None,
             crop_size=None,
             num_workers=0,
             print_detail=True)
```

In [1]:
!python PaddleSeg/val.py \
        --config PaddleSeg/configs/deeplabv3p/deeplabv3p_resnet50_os8_cityscapes_1024x512_80k.yml \
        --model_path output/iter_100000/model.pdparams 

### 模型预测

> PaddleSeg/paddleseg/core/predict.py


```
def predict(model,
            model_path,
            transforms,
            image_list,
            image_dir=None,
            save_dir='output',
            aug_pred=False,
            scales=1.0,
            flip_horizontal=True,
            flip_vertical=False,
            is_slide=False,
            stride=None,
            crop_size=None)
```
```
pred_im = utils.visualize(im_path, pred, weight=0.0)
result_saved_path = os.path.join(result_saved_dir, im_file)
mkdir(result_saved_path)
cv2.imwrite(pred_saved_path, pred_im)
```

In [14]:
!python PaddleSeg/predict.py \
       --config PaddleSeg/configs/deeplabv3p/deeplabv3p_resnet50_os8_cityscapes_1024x512_80k.yml \
       --model_path output/iter_100000/model.pdparams \
       --image_path img_testA \
       --save_dir result

In [15]:
# 由预测结果生成提交文件
!zip -r result.zip result/result/

### 相关资料

#### 飞桨领航团实战速成营：[https://aistudio.baidu.com/aistudio/education/group/info/16606](https://aistudio.baidu.com/aistudio/education/group/info/16606)
#### B站视频链接（P5、P6）：[https://www.bilibili.com/video/BV1pp4y1871g](https://www.bilibili.com/video/BV1pp4y1871g)
#### 图像分割遥感项目实战：[https://aistudio.baidu.com/aistudio/projectdetail/1703449](https://aistudio.baidu.com/aistudio/projectdetail/1703449)
#### 2020 CCF BDCI遥感地块分割解决方案：[https://aistudio.baidu.com/aistudio/projectdetail/1759079?channelType=0&channel=0](https://aistudio.baidu.com/aistudio/projectdetail/1759079?channelType=0&channel=0)
#### 2020 CCF BDCI 地块分割Top1方案 & 语义分割trick整理：[https://zhuanlan.zhihu.com/p/346862877](https://zhuanlan.zhihu.com/p/346862877)

### 比赛页面传送门： [常规赛：遥感影像地块分割](https://aistudio.baidu.com/aistudio/competition/detail/63)    欢迎一键三连 Fork~